# 1. Coleta de Dados

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as rq

In [2]:
campeonatos = []
for i in range(2007,2017):
    campeonatos.append(i)
campeonatos

[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

In [3]:
urls = []
for i in range(len(campeonatos)):
    urls.append('https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id='
                +str(campeonatos[i]))
urls

['https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2007',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2008',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2009',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2010',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2011',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2012',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2013',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2014',
 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id

In [43]:
allNames = []
allSquads = []
allAges = []
allFPs = []
allValues = []
allAvgValues = []
allRelValues = []

allPos = []
allGDs = []
allPoints = []

headers = {'User-Agent': 'ajgan'}

for a in range(len(urls)):
    r = rq.get(urls[a], headers=headers)
    parsed_page = BeautifulSoup(r.text,"html.parser")
    
    teams = parsed_page.find_all('tr')[11:31]
    positions = parsed_page.find_all('tr')[72:92]
    
    teamName = []
    teamSquad = []
    teamAge = []
    teamFP = []
    teamValue = []
    teamAvgValue = []
    teamRelValue = []

    teamPos = []
    teamGD = []
    teamPoints = []

    for i in range(len(teams)):
        teamName.append(teams[i].find_all('td')[2].text)
        teamSquad.append(int(teams[i].find_all('td')[3].text))
        teamAge.append(float(teams[i].find_all('td')[4].text.replace(",",".")))
        teamFP.append(int(teams[i].find_all('td')[5].text))
        
        value = float((teams[i].find_all('td')[6].text.replace(" Mill. €", "0000").replace(",","").replace(" Th. €", "000")))
        teamValue.append(value)
        avgValue = float((teams[i].find_all('td')[7].text.replace(" Mill. €", "0000").replace(",","").replace(" Th. €", "000")))
        teamAvgValue.append(avgValue)
        
    for j in range(len(teamName)):
        for k in range(len(positions)):
            if (teamName[j] == positions[k].find_all('a')[1].text):
                teamPos.append(int(positions[k].find_all('td')[0].text))
                teamGD.append(positions[k].find_all('td')[4].text)
                teamPoints.append(int(positions[k].find_all('td')[5].text))
                break
    
    totalLeagueValue = 0
    for m in range(len(teams)):
        teamName[m] = teamName[m] + " - " + str(campeonatos[a]+1)
        totalLeagueValue += teamValue[m]
    
    for n in range(len(teams)):
        teamRelValue.append(teamValue[n]/totalLeagueValue)
        
        
    allNames.extend(teamName)
    allSquads.extend(teamSquad)
    allAges.extend(teamAge)
    allFPs.extend(teamFP)
    allValues.extend(teamValue)
    allAvgValues.extend(teamAvgValue)
    allRelValues.extend(teamRelValue)

    allPos.extend(teamPos)
    allGDs.extend(teamGD)
    allPoints.extend(teamPoints)

In [44]:
columns = ['Equipe', 'Pos', 'Pts', 'SG', 'TamanhoElenco', 'IdadeElenco', 'JogadoresEstrangeiros','Valor', 'ValorMedio', 'ValorRelativo']
brasileirao = pd.DataFrame(list(zip(allNames, allPos, allPoints, allGDs, allSquads, allAges, allFPs, allValues, allAvgValues, allRelValues))
                          , columns=columns)

In [45]:
brasileirao

,Equipe,Pos,Pts,SG,TamanhoElenco,IdadeElenco,JogadoresEstrangeiros,Valor,ValorMedio,ValorRelativo
0,São Paulo - 2008,1,75,30,41,24.2,1,54150000.0,1320000.0,0.298232
1,Palmeiras - 2008,4,65,10,28,23.4,1,23600000.0,843000.0,0.129977
2,Flamengo - 2008,5,64,20,30,23.5,1,18500000.0,617000.0,0.101889
3,Internacional - 2008,6,54,1,34,23.3,4,18000000.0,529000.0,0.099135
4,Grêmio - 2008,2,72,24,34,22.7,1,11900000.0,350000.0,0.065539
5,Fluminense - 2008,14,45,1,35,23.2,1,8700000.0,249000.0,0.047915
6,Portuguesa - 2008,19,38,-22,33,24.8,1,6150000.0,186000.0,0.033871
7,Santos FC - 2008,15,45,-9,29,22.4,3,6050000.0,209000.0,0.033320
8,Atlético-MG - 2008,12,48,-10,42,23.4,4,5540000.0,132000.0,0.030512
9,Cruzeiro - 2008,3,67,15,25,23.1,1,4950000.0,198000.0,0.027262


In [46]:
brasileirao.to_csv("brasileirao.csv")